In [21]:
from dlinputs import tarrecords
reload(tarrecords)

<module 'dlinputs.tarrecords' from 'dlinputs/tarrecords.py'>

In [22]:
stream = open("testdata/imagenet-000000.tgz")
data = tarrecords.tardata(stream)
samples = list(data)
assert samples[0] == ('10.cls', '304'), samples[0]
assert {2} == set([len(x) for x in samples])

In [23]:
stream = open("testdata/imagenet-000000.tgz")
data = tarrecords.tardata(stream)
data = tarrecords.group_by_keys()(data)
samples = list(data)
print samples[0].keys()

['wnid', 'xml', 'png', '__key__', 'cls']


In [24]:
!tar -ztvf testdata/imagenet-000000.tgz | sed 7q
!tar xvf testdata/imagenet-000000.tgz 10.png
!file 10.png

-rw-r--r-- tmb/tmb           3 1969-12-31 16:00 10.cls
-rw-rw-r-- tmb/tmb       75442 2018-04-16 10:21 10.png
-rw-r--r-- tmb/tmb           9 1969-12-31 16:00 10.wnid
-rw-r--r-- tmb/tmb           4 1969-12-31 16:00 10.xml
-rw-r--r-- tmb/tmb           3 1969-12-31 16:00 12.cls
-rw-rw-r-- tmb/tmb       80108 2018-04-16 10:21 12.png
-rw-r--r-- tmb/tmb           9 1969-12-31 16:00 12.wnid
tar: write error
10.png
10.png: PNG image data, 600 x 793, 8-bit/color RGB, non-interlaced


In [25]:
stream = open("testdata/imagenet-000000.tgz")
data = tarrecords.tardata(stream)
data = tarrecords.group_by_keys()(data)
data = tarrecords.decoder()(data)
samples = list(data)
print samples[0].keys()

['wnid', 'xml', '__key__', 'png', 'cls']


In [26]:
stream = open("testdata/imagenet-000000.tgz")
data = tarrecords.tariterator1(stream)
samples = list(data)
assert len(samples)==47
assert samples[0]["__key__"] == "10", samples[0]["__key__"]
assert set(samples[3].keys()) == set("__key__ png cls xml wnid".split()), samples[3].keys()
assert samples[-1]["png"].shape == (400, 300, 3)

In [14]:
stream = open("testdata/imagenet-000000.tgz")
data = tarrecords.tariterator(stream)
samples = list(data)
assert len(samples)==47
assert samples[0]["__key__"] == "10"
assert set(samples[3].keys()) == set("__key__ cls png xml wnid".split())
assert samples[-1]["png"].shape == (400, 300, 3)

In [15]:
stream = open("/tmp/test.tgz", "wb")
sink = tarrecords.TarWriter(stream)
for sample in samples:
    sink.write(sample)
sink.close()
stream.close()
!ls -l /tmp/test.tgz

-rw-rw-r-- 1 tmb tmb 12993102 Apr 16 10:23 /tmp/test.tgz


In [20]:
stream = open("/tmp/test.tgz")
data = tarrecords.tariterator(stream)
samples = list(data)
assert len(samples)==47
assert samples[0]["__key__"] == "10"
assert set(samples[3].keys()) == set("__key__ cls png xml wnid".split()), samples[3].keys()
assert samples[-1]["png"].shape == (400, 300, 3)